In [1]:
import pandas as pd
import requests
import time
import json
df=pd.read_csv('../data/activity_counts.csv')

You may change your own github_token in the config.py. You can just refer here https://github.com/settings/tokens. click `personal access tokens` --> `generate new token`

In [2]:
from config import github_token

In [3]:
github_token

'8c45fcf62854847cc6f320172e5cf31be9bdc974'

In [4]:
df

Course_Code    Student_Name                          github_link  \
0          NaN         Hu Pili            https://github.com/hupili   
1          NaN        Xu Yucan          https://github.com/ChicoXYC   
2     COMM7780        GAO Chao        https://github.com/FLYSTEPHEN   
3     COMM7780       LI Chenxi        https://github.com/ConnorLi96   
4     JOUR7280         YAO Lan        https://github.com/lullabymia   
5     JOUR7280      WANG Ziwei        https://github.com/ivywang958   
6     COMM7780      LIN Yitong          https://github.com/graceyit   
7     JOUR7280    ZHAI Menghan         https://github.com/Zmh951010   
8     COMM7780     ZENG Sining       https://github.com/CelineTsang   
9     COMM7780      ZHANG Ning          https://github.com/iiiJenny   
10    COMM7780      FENG Yujin       https://github.com/Catmint1215   
11    JOUR7280          XU Wen     https://github.com/warrior960812   
12    COMM7780       HUANG Xin          https://github.com/Rita0719   
13    COMM7780        LIU Cong     https://github.com/clairelau0108   
14    JOUR7280   ZHANG Jingwei  https://github.com/zhangjingwei0512   
15    COMM7780       XIE Jieyi           https://github.com/JIEYI22   
16    JOUR7280       XU Kaiwen        https://github.com/kaiwenxu94   
17    COMM7780    ZENG Huashan   https://github.com/AlexZenghuashan   
18    COMM7780      XU Haotian        https://github.com/coolxu8888   
19    COMM7780     ZHANG Shiyu    https://github.com/CathyChang1996   
20    COMM7780          LUO Yi       https://github.com/hakutoccino   
21    JOUR7280         YU Qing          https://github.com/18426425   
22    COMM7780         CHEN Ye        https://github.com/Shinonomee   
23    JOUR7280    ZHANG Huimin     https://github.com/ZhangHuimin97   
24    JOUR7280    JIANG Nixuan        https://github.com/maggie0114   
25    COMM7780     LIN Huiqing          https://github.com/lalaLynn   
26    JOUR7280    ZHANG Zichun         https://github.com/NicoleZZC   
27    JOUR7280        LI Yijie          https://github.com/Alana-LI   
28    COMM7780      ZHANG Ning     https://github.com/ZhangNingNina   
29    COMM7780  HUANG Yangming         https://github.com/Wong-Ming   
30    JOUR7280      SHI Yuning     https://github.com/shiyuning0987   
31    COMM7780     CAI Xuejiao         https://github.com/PepperCai   
32         NaN  Huang Qiongyao     https://github.com/SerenaQYHuang   
33    COMM7780    ZENG Xingjie       https://github.com/zacharyzeng   
34    JOUR7280       HE Wenshu    https://github.com/elusiveburglar   
35    COMM7780      HUANG Ting         https://github.com/trista520   
36    COMM7780       DU Jiawei      https://github.com/jasmine-dudu   
37    JOUR7280        YAO Nuan       https://github.com/yaonuan1995   
38    JOUR7280     LI Shengnan       https://github.com/CCCCCecilia   
39    JOUR7280       LI Mengyu            https://github.com/ClioLI   
40    COMM7780    CHEN Mengjie          https://github.com/DidoChan   
41    JOUR7280        YANG Lin          https://github.com/Yang9305   
42    COMM7780        GU Minyi           https://github.com/guminyi   
43    JOUR7280     ZHANG Bohan          https://github.com/ZHANGboh   
44    JOUR7280     JIANG Weiqi            https://github.com/efncij   
45         NaN  Cheng Yuanyuan           https://github.com/niegung   

          remark  data-count         github_id  
0       lecturer        1001            hupili  
1             TA         534          ChicoXYC  
2        student         392        FLYSTEPHEN  
3   class helper         204        ConnorLi96  
4   class helper         160        lullabymia  
5        student         143        ivywang958  
6   class helper         139          graceyit  
7   class helper         102         Zmh951010  
8   class helper          95       CelineTsang  
9   class helper          95          iiiJenny  
10  class helper          85       Catmint1215  
11  class helper          78     warrior960812  
12       student          73

In [5]:
users = df['github_id'].tolist()

In [6]:
# only can get 300 maximum 
def get_details(url):
    r = requests.get(url,headers={
    'User-Agent':'Mozilla/5.0',
    'Authorization':'token {0}'.format(github_token),
    'Content-Type':'application/json',
    'method':'GET',
    'Accept':'application/json'
    })
    content = r.json()
    json_str = json.dumps(content)
    
    with open('../user-activities-json/{0}-page{1}&per_page=100.json'.format(user,i), "w") as f:
        f.write(json_str)
        f.close()

    commits = []
    for item in content:
        commit = {}
        commit['time'] = item['created_at']
        commit['repo'] = item['repo']['name']
        commit['users'] = item['actor']['login']
        commit['type'] = item['type']
        if item['type'] == 'CreateEvent':
            commit['description'] = item['repo']['url']
        elif item['type'] == 'IssuesEvent':
            commit['description'] = item['payload']['action']
        elif item['type'] == 'PushEvent':
            commit['description'] = item['repo']['url']
        elif item['type'] == 'WatchEvent':
            commit['description'] = item['repo']['url']
        else:
            commit['description'] = None
        
        commits.append(commit)
    return commits

In [7]:
all_commits = []
for user in users:
    one_user_commits=[]
    for i in range(1,4):
        url = 'https://api.github.com/users/{0}/events/public?page={1}&per_page=100'.format(user,i)
        commits = get_details(url)
        one_user_commits.extend(commits)
    all_commits.extend(one_user_commits)

In [8]:
df2 = pd.DataFrame(all_commits)

In [9]:
import csv
with open('../data/user-activity-details.csv','a') as f:
    df2.to_csv(f, header=None,index=None)

In [10]:
df3 = pd.read_csv('../data/user-activity-details.csv')

In [11]:
len(df3)

6451

In [12]:
df3 = df2.drop_duplicates()

In [13]:
len(df3)

3386

In [14]:
df3.to_csv('../data/user-activity-details.csv',index=None)

In [15]:
df4 = df3.pivot_table(index=['users','type'],values = 'time',aggfunc='count')
df4

time
users            type                    
18426425         CommitCommentEvent     2
                 CreateEvent           12
                 DeleteEvent            1
                 ForkEvent              2
                 IssueCommentEvent      3
                 IssuesEvent            1
                 PushEvent             33
                 WatchEvent             6
Alana-LI         CommitCommentEvent     1
                 CreateEvent            6
                 ForkEvent              2
                 PushEvent             35
AlexZenghuashan  CreateEvent           11
                 ForkEvent              1
                 IssueCommentEvent      6
                 IssuesEvent            5
                 PushEvent             32
CCCCCecilia      CreateEvent            4
                 ForkEvent              1
                 IssueCommentEvent      6
                 IssuesEvent            3
                 PushEvent             40
                 WatchEvent             2
CathyChang1996   CreateEvent            8
                 ForkEvent              1
                 IssueCommentEvent      9
                 IssuesEvent            5
                 PushEvent             48
Catmint1215      CreateEvent           16
                 ForkEvent              3
...                                   ...
maggie0114       ForkEvent              1
                 PushEvent             29
shiyuning0987    CreateEvent            5
                 ForkEvent              1
                 IssueCommentEvent      3
                 IssuesEvent            1
                 PushEvent             22
trista520        CreateEvent            8
                 ForkEvent              2
                 PushEvent             13
warrior960812    CreateEvent           12
                 ForkEvent              1
                 IssueCommentEvent      1
                 PullRequestEvent       1
                 PushEvent             78
                 WatchEvent             1
yaonuan1995      CreateEvent            2
                 ForkEvent              1
                 IssueCommentEvent      4
                 IssuesEvent            2
                 PushEvent             16
zacharyzeng      CreateEvent            8
                 DeleteEvent            1
                 ForkEvent              1
                 IssueCommentEvent      2
                 IssuesEvent            1
                 PushEvent             44
zhangjingwei0512 CreateEvent           14
                 IssueCommentEvent      1
                 PushEvent             18

[234 rows x 1 columns]